In [ ]:
!pip install keras
!pip install tensorflow

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import tensorflow as tf
import random
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import roc_auc_score
from keras.layers import Dense, LeakyReLU

train = pd.read_csv('input/train.csv') # train data link
test = pd.read_csv('input/test.csv') # test data link

# Data preprocessing
### Delete duplicates and Check for unique values in each features

In [2]:
def scan_dup(df, delete = False):
    # Print number of duplicates for each line , duplicates deleted when 'delete' be True.
    print(f"# of duplicates rows : {df.duplicated().sum()}, ({np.round(100*df.duplicated().sum()/len(df),1)}%)")
    if delete :
        df.drop_duplicates(inplace = True)

def remove_badfeat(df_train, df_test, features):
    # Remove categorical features where train&test dataframe have different unique values
    for feat in features :
        if len(set(df_train[feat].unique()) ^ set(df_test[feat].unique()) )> 0 :
            print(f"Cat-feature {feat} has different values in train & test set \n")
            print(f" --> {feat} is deleted \n")
            print(df_train[feat].unique(), df_test[feat].unique() ,"\n")
            del df_train[feat]
            del df_test[feat]

###########
scan_dup(train)
print("\n")
train.nunique().sort_values(ascending=True)
#print("\n")
#test["product_code"].value_counts()

# of duplicates rows : 0, (0.0%)




failure               2
attribute_0           2
attribute_1           3
attribute_2           4
attribute_3           4
product_code          5
measurement_2        25
measurement_0        29
measurement_1        30
measurement_5      4671
measurement_4      4692
measurement_6      4704
measurement_9      4708
measurement_8      4713
measurement_3      4721
measurement_7      4734
measurement_13     5271
measurement_10     6177
measurement_14     6389
measurement_12     6392
measurement_11     6526
measurement_15     6577
measurement_16     7035
loading           11950
measurement_17    23612
id                26570
dtype: int64

### Check  missing values of each feature



In [3]:
train.isnull().sum()

id                   0
product_code         0
loading            250
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      381
measurement_4      538
measurement_5      676
measurement_6      796
measurement_7      937
measurement_8     1048
measurement_9     1227
measurement_10    1300
measurement_11    1468
measurement_12    1601
measurement_13    1774
measurement_14    1874
measurement_15    2009
measurement_16    2110
measurement_17    2284
failure              0
dtype: int64

In [4]:
train_gp = train.groupby('product_code').mean().T
test_gp = test.groupby('product_code').mean().T

data = pd.concat([train_gp,test_gp], axis = 1 ).corr(method = 'kendall')
data[data==1] = -1

#test['product_code'].value_counts()

C:\Users\confi\AppData\Local\Temp\ipykernel_9168\3744922203.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train_gp = train.groupby('product_code').mean().T
C:\Users\confi\AppData\Local\Temp\ipykernel_9168\3744922203.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_gp = test.groupby('product_code').mean().T


### Features filtering

In [5]:
# Those having more than 50 unique values are considered numerical features
cat_feat = [feat for feat in train.columns if train[feat].nunique() < 50 and feat!= "attribute_2" and feat!="attribute_3" and feat!= 'failure' ]
num_feat = [feat for feat in train.columns if feat not in cat_feat and feat!= 'failure']

# Remove categorical features where train&test dataframe have different unique values
remove_badfeat(train,test,cat_feat)

# Train&Test sets both have same unique values in each column , encode the former feature as a binary variable
train['attribute_0'] = train['attribute_0'].map({'material_7':0,'material_5':1})
test['attribute_0'] = test['attribute_0'].map({'material_7':0,'material_5':1})

Cat-feature product_code has different values in train & test set 

 --> product_code is deleted 

['A' 'B' 'C' 'D' 'E'] ['F' 'G' 'H' 'I'] 

Cat-feature attribute_1 has different values in train & test set 

 --> attribute_1 is deleted 

['material_8' 'material_5' 'material_6'] ['material_6' 'material_7' 'material_5'] 

Cat-feature measurement_0 has different values in train & test set 

 --> measurement_0 is deleted 

[ 7 14 12 13  9 11  4 10  6  8 21 15 17 18 19 16  5 25  3  1 23 20 22  2
 26 24  0 29 27] [ 6 11  8 14 10 16  7 20  9  5  2 13  3  4 15 19 12 22 21 18 17 23  0 26
 24  1 25 29 30 28] 

Cat-feature measurement_1 has different values in train & test set 

 --> measurement_1 is deleted 

[ 8  3  1  2  4  6  0  9  5  7 10 12 11 13 17 14 16 15 18 20 24 22 21 19
 23 27 25 26 29 28] [ 9  8 12 11 16 18  7 15 19 10 13  6 14  5  2  4 17 25 22 21 23  3 20 26
 24 31 27 28  1 29 33 32  0] 

Cat-feature measurement_2 has different values in train & test set 

 --> measurement_2 is del

### Filling missing data and label encoding categorical features

In [6]:
train['measurement_avg'] = train[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)
test['measurement_avg'] = test[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)

train['m3_missing'] = train['measurement_3'].isnull().astype(np.int8)
train['m5_missing'] = train['measurement_5'].isnull().astype(np.int8)

test['m3_missing'] = test['measurement_3'].isnull().astype(np.int8)
test['m5_missing'] = test['measurement_5'].isnull().astype(np.int8)


#measure_gp1_cols = [f"measurement_{i:d}" for i in list(range(3, 5)) + list(range(9, 17))]
measure_gp2_cols = [f"measurement_{i:d}" for i in list(range(5, 9))]
train['measure_gp2_avg'] = np.mean(train[measure_gp2_cols], axis=1)
test['measure_gp2_avg'] = np.mean(test[measure_gp2_cols], axis=1)

train['attribute_2*3'] = train['attribute_2'] * train['attribute_3']
test['attribute_2*3'] = test['attribute_2'] * test['attribute_3']

#train.head()

# One-hot encoding
encoded_col = []
for col in encoded_col:
     tmp_train= pd.get_dummies(test[col], prefix = col)
     test = pd.merge(left = test, right = tmp_train, left_index = True, right_index = True)
test = test.drop(encoded_col, axis = 1)
#test.head()

x_train = train[train.columns.difference(["failure",'id',"attribute_2","attribute_3"])]
x_test = test[test.columns.difference(['id', "attribute_2","attribute_3"])]
y_train = train["failure"]

#x_train.head()
#x_test.head()
iter_imputer = IterativeImputer(max_iter = 8, random_state = 0, skip_complete = True, n_nearest_features = 12)
x_train= iter_imputer.fit_transform(x_train)
x_test=iter_imputer.transform(x_test)

c:\Users\confi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_iterative.py:713: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


# Load model & get prediction

In [7]:
from keras.models import load_model
# Model download link : https://drive.google.com/file/d/1Jbzr1SQ-iFxKGfbqNFEFfhpfu1-Er1wH/view?usp=share_link
# Or run 109550135_Final_train.ipynb first , it'll be in the same directory as this file
model = load_model('109550135_model.h5') #   
y_pred = model.predict(x_test)

650/650 [==============================] - 0s 503us/step


In [8]:
submission = pd.read_csv('input/sample_submission.csv') # sample_submission.csv link
submission['failure'] = y_pred
submission.to_csv('109550135.csv',index=False)